# Imports and Filter Definitions

In [1]:
import pandas as pd
import numpy as np
import requests
from pprint import pprint

In [2]:
accepted_codes = ['910','911','912','913','914','915','916','917','918','919','920','921','922', \
                  '923','924','925','926','927','928','929','970','971','972','973','974','975']

def filter_luc(dataframe):
    """filters by land use codes affiliated with MA state agencies
    """
    return dataframe[dataframe['luc_1'].isin(accepted_codes) | \
            dataframe['luc_2'].isin(accepted_codes) | \
            dataframe['luc_adj_1'].isin(accepted_codes)| \
            dataframe['luc_adj_2'].isin(accepted_codes)]

def filter_poly_typ(dataframe):
    # filter out data only with poly_typ equal to FEE or TAX, returns pandas dataframe
    accepted_codes = ['FEE', 'TAX']
    return dataframe[dataframe['poly_typ'].isin(accepted_codes)]

def filter_bldg(dataframe):
    '''
    Filter on related columns that indicate whether building(s) are present on the land parcel.
    Removes rows that correspond to land parcels that do not contain buildings.
    Ziba specified: 
        bldg_value - for condos, generally includes land value
        bldg_area - may include garages, stairwells, basements, and other uninhabitable areas.
        bldgv_psf - building value $ per sq foot
    Additional: 
        sqm_bldg - parcel area estimated to be covered by a building (sq meters)
        pct_bldg - % parcel area estimated to be covered by a building 
    '''
    
    return dataframe.query('bldg_value > 0 | \
                           bldg_area > 0 | \
                           bldgv_psf > 0 | \
                           sqm_bldg > 0 | \
                           pct_bldg > 0')

# Test single request

In [92]:
# test single address request

URL = "https://geocoding.geo.census.gov/geocoder/geographies/address?street=4600+Silver+Hill+Rd&" + \
    "city=Suitland&state=MD&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010&layer" + \
    "s=14&format=json"
  
r = requests.get(url = URL)

data = r.json()

print(data)

{'result': {'input': {'benchmark': {'id': '9', 'benchmarkName': 'Public_AR_Census2010', 'benchmarkDescription': 'Public Address Ranges - Census 2010 Benchmark', 'isDefault': False}, 'vintage': {'id': '910', 'vintageName': 'Census2010_Census2010', 'vintageDescription': 'Census2010 Vintage - Census2010 Benchmark', 'isDefault': True}, 'address': {'street': '4600 Silver Hill Rd', 'city': 'Suitland', 'state': 'MD'}}, 'addressMatches': [{'matchedAddress': '4600 Silver Hill Rd, SUITLAND, MD, 20746', 'coordinates': {'x': -76.92691, 'y': 38.846542}, 'tigerLine': {'tigerLineId': '613199520', 'side': 'L'}, 'addressComponents': {'fromAddress': '4600', 'toAddress': '4712', 'preQualifier': '', 'preDirection': '', 'preType': '', 'streetName': 'Silver Hill', 'suffixType': 'Rd', 'suffixDirection': '', 'suffixQualifier': '', 'city': 'SUITLAND', 'state': 'MD', 'zip': '20746'}, 'geographies': {'Census Blocks': [{'SUFFIX': '', 'POP100': 0, 'GEOID': '240338024051084', 'CENTLAT': '+38.8477828', 'BLOCK': '108

In [139]:
# test single address request

URL = "https://geocoding.geo.census.gov/geocoder/geographies/address?street=4600+Silver+Hill+Rd&" + \
    "&state=MD&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010&layer" + \
    "s=14&format=json"
  
r = requests.get(url = URL)

data = r.json()

print(data)

{'errors': ['Specify street with city and state or zip']}


In [99]:
pprint(data)

{'result': {'addressMatches': [{'addressComponents': {'city': 'SUITLAND',
                                                      'fromAddress': '4600',
                                                      'preDirection': '',
                                                      'preQualifier': '',
                                                      'preType': '',
                                                      'state': 'MD',
                                                      'streetName': 'Silver '
                                                                    'Hill',
                                                      'suffixDirection': '',
                                                      'suffixQualifier': '',
                                                      'suffixType': 'Rd',
                                                      'toAddress': '4712',
                                                      'zip': '20746'},
                                'co

In [116]:
tract = data['result']['addressMatches'][0]['geographies']['Census Blocks'][0]['TRACT']
print(tract)

802405


# Single Request Function Definition

In [126]:
# test clean-up of street address
addr_str = ' oak maplewood road '

street_split = addr_str.split()
addr_str_formatted = ''
# remove extra white spaces
for i in range(len(street_split)):
    if (not street_split[i].isspace()):
        addr_str_formatted = addr_str_formatted + street_split[i]
        
        if (i != len(street_split)-1):
            addr_str_formatted = addr_str_formatted + '+'
        
print('\\' + addr_str_formatted + '\\')

\oak+maplewood+road\


In [14]:
def get_tract_number(addr_num, addr_str, city=None, addr_zip=None):
    '''
        Makes a REST API request to Census Geocoder API and returns a tract number for a given address.
        Inputs:
            - addr_num = street number, can be a range of street numbers, e.g. 200-400
            - addr_str = street name
            - addr_zip = zip code
            - city = city/town name
        Note: addr_num, addr_str, and addr_zip OR (city/town + state) are required fields! If any are unavailable, then the request fails.
            This is despite the API documentation stating that zip code is an optional field.
    '''
    # remove any extra white space in the addr_num field
    addr_num = str(addr_num).replace(" ", "")
    addr_str = addr_str.replace('#', "")
    
    # join separate words in street name with '+'
    street_split = addr_str.split()
    addr_str_formatted = ''
    # remove extra white spaces
    for i in range(len(street_split)):
        if (not street_split[i].isspace()):
            addr_str_formatted = addr_str_formatted + street_split[i]

            if (i != len(street_split)-1):
                addr_str_formatted = addr_str_formatted + '+'
    
    URL = "https://geocoding.geo.census.gov/geocoder/geographies/address?street=" + addr_num + "+" \
        + addr_str_formatted
    
    if (addr_zip is not None and addr_zip):
        URL = URL + "&zip=" + str(addr_zip).replace(" ", "")
    else: # there shouldn't be any blanks in the city/town field (muni in land_parcel db)
        URL = URL + "&city=" + str(city).strip() + "&state=MA"
    
    URL = URL + "&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010" + \
        "&layers=14&format=json"
    
    #print("URL: ", URL)

    response = requests.get(url = URL)
    tract = np.nan
    
    if (response.status_code == 200):
        data = response.json()
        # pprint(data)
        if (len(data['result']['addressMatches']) > 0):
            # if data['result']['addressMatches'][0]['geographies']['Census Blocks'][0]['TRACT']:
            tract = data['result']['addressMatches'][0]['geographies']['Census Blocks'][0]['TRACT']
    
    return tract

In [13]:
# test function

print(get_tract_number(200, 'falls blvd', addr_zip='02169 '))
print(get_tract_number('200', 'falls blvd ', addr_zip='02169'))
print(get_tract_number('200-400', 'falls blvd', addr_zip='02169'))

URL:  https://geocoding.geo.census.gov/geocoder/geographies/address?street=200+falls+blvd&zip=02169&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010&layers=14&format=json
418002
URL:  https://geocoding.geo.census.gov/geocoder/geographies/address?street=200+falls+blvd&zip=02169&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010&layers=14&format=json
418002
URL:  https://geocoding.geo.census.gov/geocoder/geographies/address?street=200-400+falls+blvd&zip=02169&benchmark=Public_AR_Census2010&vintage=Census2010_Census2010&layers=14&format=json
418002


# Batch Request Code & Testing Below (unsuccessful)

In [4]:
def get_geocode_info(dataframe):
    '''
    Makes a batch GET request to Census Geocoder API.
    Dataframe must include street address.
    Optional fields: City, State, ZIP
    '''
    
    def format_csv(dataframe):
        '''
        Helper function that takes pandas dataframe list of addresses and converts it to appropriate csv format
        necessary for REST API request.

        Format columns as following:
        Unique ID, Street address, City, State, ZIP
        where City, State, ZIP are optional fields but file must contain a placeholder comma if left blank.
        '''

        # land parcel data only has addr_num, addr_str, and addr_zip fields
        df = land_parcel_df[['addr_num', 'addr_str', 'addr_zip']]

        # copy over indices to be unique id column
        df['Unique ID'] = dataframe.index

        # drop rows where addr_num or addr_str are null
        df.dropna(subset=['addr_num', 'addr_str'], inplace=True)

        # concatenate addr_num and addr_str into single column
        df['Street address'] = df['addr_num'].astype(str) + ' ' + df['addr_str']

        # add empty columns for city and state, so that csv file is correctly formatted with commas for blanks
        df['City'] = ''
        df['State'] = ''
        df['ZIP'] = df['addr_zip']

        # replace nans in ZIP with blanks
        df[df['addr_zip'].isnull()] = ''

        df.drop(columns=['addr_num', 'addr_str', 'addr_zip'])

        df = df[['Unique ID', 'Street address', 'City', 'State', 'ZIP']]

        # drop rows that are completely blank
        df.dropna(how='all')

        df.to_csv('geocode_format.csv', index=False, header=False)
    
    # creates formatted csv file that gets saved to current directory as 'geocode_format.csv'
    format_csv(dataframe)
    csv_file = open('geocode_format.csv', 'r')
    
    # geographies contains the State, County, Tract, and Block layers by default
    returntype = 'geographies' # locations (just geocoding response) or geographies (geocoding response + geoLookup)
    BATCH_GEOCODE_URL = "https://geocoding.geo.census.gov/geocoder/" + returntype + "/addressbatch"

    PARAMS = {'returntype': returntype, \
              'benchmark': 'Public_AR_Census2010', \
              'vintage': 'Census2010_Census2010', \
             'addressFile': csv_file} 
    
    r = requests.get(url = URL, params = PARAMS)

    output_csv = open('geocode_result.csv', 'w')
    output_csv.write(r.text)
    output_csv.close()

In [80]:
import httplib2

h = httplib2.Http()

csv_file = open('test.csv')

# geographies contains the State, County, Tract, and Block layers by default
returntype = 'geographies' # locations (just geocoding response) or geographies (geocoding response + geoLookup)
BATCH_GEOCODE_URL = "https://geocoding.geo.census.gov/geocoder/" + returntype + "/addressbatch"

PARAMS = {'benchmark': 'Public_AR_Census2010', \
            'vintage': 'Census2010_Census2010'} 

resp, content = h.request(BATCH_GEOCODE_URL, \
                          method="GET", \
                          body = csv_file, \
                          headers = PARAMS)

print(resp.status)
print(resp.reason)

400
Bad Request


In [82]:
csv_file = open('test.csv', 'r')
    
# geographies contains the State, County, Tract, and Block layers by default
returntype = 'geographies' # locations (just geocoding response) or geographies (geocoding response + geoLookup)
BATCH_GEOCODE_URL = "https://geocoding.geo.census.gov/geocoder/" + returntype + "/addressbatch"

PARAMS = {'benchmark': 'Public_AR_Census2010', \
          'vintage': 'Census2010_Census2010', \
         'addressFile': csv_file} 

r = requests.get(url = URL, params = PARAMS)

print(r)

# output_csv = open('geocode_result.csv', 'w')
# output_csv.write(r.text)
# output_csv.close()

<Response [400]>


# Main program

In [15]:
df = pd.read_csv('../../data/land_parcel/mapc.ma_parcels_metrofuture.csv', \
                 dtype={'addr_zip': np.string_}) # zip codes need to be imported as string in order to retain leading 0

land_parcel_df = filter_luc(df)
land_parcel_df = filter_poly_typ(land_parcel_df)
# land_parcel_df = filter_bldg(land_parcel_df) # filter on top of the filtered table

# land_parcel_df = pd.read_csv('../../data/filtered_data/usable_state_land.csv', \
#                   dtype={'addr_zip': np.string_}) # zip codes need to be imported as string in order to retain leading 0

C:\Users\xsmil\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6,14,17,18,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
# reset indices in order to refer to rows by index in for loop below
# keeps original index values in new column 'index'
land_parcel_df = land_parcel_df.reset_index()

In [19]:
print(land_parcel_df.at[31, 'addr_num'], \
land_parcel_df.at[31, 'addr_str'], \
land_parcel_df.at[31, 'addr_zip'])

num = land_parcel_df.at[31, 'addr_num']
street = land_parcel_df.at[31, 'addr_str']
zipcode = land_parcel_df.at[31, 'addr_zip']

print(get_tract_number(num, street, zipcode))

54 MEDFORD ST 02476
nan


In [18]:
land_parcel_df[['muni', 'addr_num', 'addr_str', 'addr_zip']]

,muni,addr_num,addr_str,addr_zip
0,Abington,NaN,BIRCH ST,02351
1,Abington,NaN,BIRCH ST,02351
2,Abington,NaN,MONROE ST,02351
3,Abington,NaN,GLINIEWICZ WAY,02351
4,Abington,NaN,LINCOLN BLV,02351
...,...,...,...,...
7714,Rockport,NaN,GOTT AVE,NaN
7715,Rockport,NaN,GRANITE ST & GOTT AVE,NaN
7716,Rockport,NaN,REAR RAILROAD AVE,NaN
7717,Rockport,NaN,MILK ISLAND,NaN


In [17]:
# from datetime import datetime

# land_parcel_df['census_tract'] = np.nan

# for i in range(3200):
#     # in order to make request, need street address (number & street name), AND zip code
#     addr_num = str(land_parcel_df.iloc[i]['addr_num']).replace(" ", "")
#     addr_str = land_parcel_df.iloc[i]['addr_str']
#     addr_zip = str(land_parcel_df.iloc[i]['addr_zip']).replace(" ", "")
    
#     #print(i, addr_num, addr_str, addr_zip)
#     if (i % 100 == 0):
#         print('Step: ', i)
#         print("Current Time =", datetime.now().strftime("%H:%M:%S"))
    
#     # check for any blanks or nan values
#     if not ((not addr_num or addr_num == 'nan') \
#         or (not addr_str or addr_str == 'nan') \
#         or (not addr_zip or addr_zip == 'nan')):
#             tract = get_tract_number(addr_num, addr_str, addr_zip)
#             land_parcel_df.at[i, 'census_tract'] = tract

Step:  0
Current Time = 16:32:51
Step:  100
Current Time = 16:33:50
Step:  200
Current Time = 16:34:25
Step:  300
Current Time = 16:35:12
Step:  400
Current Time = 16:35:38
Step:  500
Current Time = 16:35:50
Step:  600
Current Time = 16:36:43
Step:  700
Current Time = 16:37:01
Step:  800
Current Time = 16:37:39
Step:  900
Current Time = 16:38:31
Step:  1000
Current Time = 16:39:12
Step:  1100
Current Time = 16:39:19
Step:  1200
Current Time = 16:39:20
Step:  1300
Current Time = 16:39:59
Step:  1400
Current Time = 16:40:32
Step:  1500
Current Time = 16:41:00
Step:  1600
Current Time = 16:41:15
Step:  1700
Current Time = 16:42:12
Step:  1800
Current Time = 16:43:14
Step:  1900
Current Time = 16:43:29
Step:  2000
Current Time = 16:43:55
Step:  2100
Current Time = 16:44:26
Step:  2200
Current Time = 16:45:02
Step:  2300
Current Time = 16:45:51
Step:  2400
Current Time = 16:46:53
Step:  2500
Current Time = 16:47:52
Step:  2600
Current Time = 16:48:43
Step:  2700
Current Time = 16:49:14
Step

In [17]:
from datetime import datetime

land_parcel_df['census_tract'] = np.nan

In [20]:
for i in range(len(land_parcel_df)):
    # in order to make request, need street address (number & street name), AND zip code
    addr_num = str(land_parcel_df.iloc[i]['addr_num']).replace(" ", "")
    addr_str = land_parcel_df.iloc[i]['addr_str']
    muni = str(land_parcel_df.iloc[i]['muni']).strip()
    addr_zip = str(land_parcel_df.iloc[i]['addr_zip']).replace(" ", "")

    #print(i, addr_num, addr_str, addr_zip)
    if (i % 100 == 0):
        print('Step: ', i)
        print("Current Time =", datetime.now().strftime("%H:%M:%S"))

    # check for any blanks or nan values
    if not ((not addr_num or addr_num == 'nan') \
        or (not addr_str or addr_str == 'nan') \
        or (not addr_zip or addr_zip == 'nan') \
        or (not muni or muni == 'nan')):
            try:
                tract = get_tract_number(addr_num, addr_str, muni, addr_zip)
                land_parcel_df.at[i, 'census_tract'] = tract
            except:
                print('Error at step: ', i)

Step:  0
Current Time = 21:39:12
Step:  100
Current Time = 21:40:01
Step:  200
Current Time = 21:40:12
Step:  300
Current Time = 21:41:01
Step:  400
Current Time = 21:41:23
Step:  500
Current Time = 21:42:06
Step:  600
Current Time = 21:42:29
Step:  700
Current Time = 21:43:13
Step:  800
Current Time = 21:43:14
Step:  900
Current Time = 21:43:19
Step:  1000
Current Time = 21:43:28
Step:  1100
Current Time = 21:43:48
Step:  1200
Current Time = 21:44:41
Step:  1300
Current Time = 21:45:09
Step:  1400
Current Time = 21:45:09
Step:  1500
Current Time = 21:45:31
Step:  1600
Current Time = 21:45:34
Step:  1700
Current Time = 21:46:26
Step:  1800
Current Time = 21:47:09
Step:  1900
Current Time = 21:47:22
Step:  2000
Current Time = 21:48:15
Step:  2100
Current Time = 21:49:04
Step:  2200
Current Time = 21:49:49
Step:  2300
Current Time = 21:49:49
Step:  2400
Current Time = 21:49:55
Step:  2500
Current Time = 21:50:11
Step:  2600
Current Time = 21:50:11
Step:  2700
Current Time = 21:50:13
Step

In [21]:
land_parcel_df.loc[land_parcel_df['census_tract'].notna(), 'census_tract']

5       520202.0
7       520202.0
8       266300.0
9       266300.0
10      266300.0
          ...   
7369    408101.0
7370    408102.0
7372    408102.0
7373    408102.0
7695    367200.0
Name: census_tract, Length: 1411, dtype: float64

# Obtain Median Household Income Estimate by Census Tract for MA

In [22]:
def get_median_hh_income():
    '''
    Returns Pandas DataFrame representation Median Household Income Estimate by Census Tract for MA.
    American Community Survey (ACS) 2018 Census data used.
    Specific table: ACS 2018 5-year detailed table "B19013_001E"
    '''
    URL = "https://api.census.gov/data/2018/acs/acs5?get=B19013_001E&for=tract:*&in=state:25"
  
    response = requests.get(url = URL)
    data = response.json()

    # pprint(data)
    
    median_income_df = pd.DataFrame(data[1:len(data)-1], columns = data[0])
    
    return median_income_df

In [23]:
median_income_df = get_median_hh_income()

In [24]:
median_income_df.head()

,B19013_001E,state,county,tract
0,132750,25,025,060501
1,12759,25,025,061101
2,84083,25,025,070101
3,28851,25,025,070402
4,52676,25,025,071101


In [25]:
median_income_df.to_csv('ma_med_income_tract.csv',index=False)

# Add Median HH Income Data to land_parcel_df by comparing tract number

In [14]:
land_parcel_df.head()

,index,objectid,mapc_id,muni_id,muni,parloc_id,poly_typ,map_num,mappar_id,loc_id_cnt,...,bldgv_psf,totv_pac,bldlnd_rat,sqm_imperv,sqm_bldg,sqm_pave,realesttyp,temp,shape,census_tract
0,1385,1366,1366,1,Abington,M_246267_875289,FEE,53,53_79,1.0,...,-0.9999,256029.052883,0.0,196.812311,-0.9999,0.0,12.0,NaN,3600000004000000080010006A6900001E030000010000...,NaN
1,1427,1407,1407,1,Abington,M_246286_875131,FEE,53,53_72,1.0,...,-0.9999,35033.119438,0.0,11249.740307,-0.9999,0.0,12.0,NaN,620000000B000000080010006A6900008A010000010000...,NaN
2,1456,1437,1437,1,Abington,M_246358_875203,FEE,54,54_113,1.0,...,-0.9999,58710.695187,0.0,11.610459,-0.9999,0.0,12.0,NaN,4800000007000000080010006A69000030020000010000...,NaN
3,2137,2115,2115,1,Abington,M_245207_874239,FEE,38,38_17,1.0,...,-0.9999,47599.635542,0.0,150.201817,-0.9999,0.0,0.0,NaN,4A00000007000000080010006A69000032010000010000...,NaN
4,2199,2178,2178,1,Abington,M_244997_874062,FEE,38,38_18,1.0,...,-0.9999,12345.548473,0.0,-0.999900,-0.9999,0.0,0.0,NaN,4A00000007000000080010006A69000032030000010000...,NaN


In [26]:
median_income_df[median_income_df['tract'] == tract]['B19013_001E']

Series([], Name: B19013_001E, dtype: object)

In [48]:
from statistics import mean

# only check for land parcels that we were able to obtain tract numbers for
for i in land_parcel_df[land_parcel_df['census_tract'].notna()].index:
    tract = land_parcel_df.loc[i]['census_tract']
    median_hh_income = median_income_df[median_income_df['tract'] == str(int(tract))]['B19013_001E']
    
    # ACS data is broken down by state > county > tract
    # sometimes tract covered more than 1 county
    # averaged the median incomes
    # print(len(median_hh_income))
    if (len(median_hh_income) > 1):
        median_hh_income = mean([int(m) for m in median_income_df[median_income_df['tract'] == tract]['B19013_001E'].values])
    
    try:
        land_parcel_df.at[i, 'median_hh_income'] = median_hh_income
    except:
        print('Error at index: ', i)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [49]:
land_parcel_df.loc[land_parcel_df['median_hh_income'].notna(), 'median_hh_income']

5        89978.0
7        89978.0
8        70063.0
9        70063.0
10       70063.0
          ...   
7369    136741.0
7370    105163.0
7372    105163.0
7373    105163.0
7695    157923.0
Name: median_hh_income, Length: 1411, dtype: float64

In [50]:
land_parcel_df.to_csv('state_land_plus_med_income.csv',index=False)